In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [30]:
data=pd.read_csv("C:\\Users\\Moin\\Desktop\\Mental Health Project\\train.csv")
data.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [31]:
data.shape

(27481, 2)

In [32]:
emotion_mapping = {'positive': 1, 'neutral': 2, 'negative': 0}

In [33]:
data['sentiment'] = data['sentiment'].map(emotion_mapping)

In [34]:
data.head()

,text,sentiment
0,"I`d have responded, if I were going",2
1,Sooo SAD I will miss you here in San Diego!!!,0
2,my boss is bullying me...,0
3,what interview! leave me alone,0
4,"Sons of ****, why couldn`t they put them on t...",0


In [35]:
data['sentiment'].value_counts()

sentiment
2    11118
1     8582
0     7781
Name: count, dtype: int64

In [36]:
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [37]:
# Download NLTK resources (if not already downloaded)
nltk.download('stopwords')

stopwords_set = set(stopwords.words('english'))
emoji_pattern = re.compile('(?::|;|=)(?:-)?(?:\)|\(|D|P)')

def preprocessing(text):
    

    # Remove HTML tags
    text = re.sub('<[^>]*>', '', text)
    
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Extract emojis
    emojis = emoji_pattern.findall(text)
    
    # Remove non-alphanumeric characters, convert to lowercase, and join emojis
    text = re.sub('[\W+]', ' ', text.lower()) + ' '.join(emojis).replace('-', '')
    
   
    # Apply stemming and remove stopwords
    prter = PorterStemmer()
    text = [prter.stem(word) for word in text.split() if word not in stopwords_set]

    return " ".join(text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Moin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
preprocessing('this is im my tags <h1> :) <p>helo world<p> <div> <div> </h2>')

'im tag helo world :)'

In [41]:
print(data['text'].isnull().sum())

1


In [42]:
data['text'] = data['text'].fillna('')

In [43]:
data['text'] = data['text'].apply(lambda x: preprocessing(x))

In [44]:
data['text']

0                                               respond go
1                                  sooo sad miss san diego
2                                               boss bulli
3                                      interview leav alon
4                            son put releas alreadi bought
                               ...                        
27476    wish could come see u denver husband lost job ...
27477    wonder rake client made clear net forc dev lea...
27478    yay good enjoy break probabl need hectic weeke...
27479                                                worth
27480                           flirt go atg smile yay hug
Name: text, Length: 27481, dtype: object

In [45]:
data.head()

,text,sentiment
0,respond go,2
1,sooo sad miss san diego,0
2,boss bulli,0
3,interview leav alon,0
4,son put releas alreadi bought,0


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None,use_idf=True,norm='l2',smooth_idf=True)
y=data.sentiment.values
x=tfidf.fit_transform(data.text)

In [48]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,random_state=1,test_size=0.2,shuffle=False)

In [49]:
from sklearn.linear_model import LogisticRegressionCV
clf=LogisticRegressionCV(cv=6,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=500).fit(X_train,y_train)
y_pred = clf.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  2.1min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  2.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  2.2min finished


In [50]:
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6821902856103329


In [51]:
import pickle
pickle.dump(clf,open('clf.pkl','wb'))
pickle.dump(tfidf,open('tfidf.pkl','wb'))

In [56]:
def prediction(comment):
    preprocessed_comment = preprocessing(comment)
    print("Preprocessed Comment:", preprocessed_comment)
    comment_list = [preprocessed_comment]
    comment_vector = tfidf.transform(comment_list)
    print("TF-IDF Vector:", comment_vector)
    decision_scores = clf.decision_function(comment_vector)
    print("Raw Decision Scores:", decision_scores)
    predicted_class = np.argmax(decision_scores)
    
    if predicted_class == 1:
        print("Positive comment")
    elif predicted_class == 0:
        print("Negative comment")
    else:
        print("Neutral comment")

prediction('Todays lecture covered various topics related to modern art.')



Preprocessed Comment: today lectur cover variou topic relat modern art
TF-IDF Vector:   (0, 18779)	0.45127037158735867
  (0, 17958)	0.3351945123423311
  (0, 17868)	0.17957756702508904
  (0, 14824)	0.3649050600027905
  (0, 11982)	0.3975728670362003
  (0, 10711)	0.38680071141238437
  (0, 5145)	0.31687286849688573
  (0, 2459)	0.332428180945521
Raw Decision Scores: [[-0.26121038 -0.54386271  0.80507309]]
Neutral comment


In [57]:
import pickle
pickle.dump(clf,open('clf.pkl','wb'))
pickle.dump(tfidf,open('tfidf.pkl','wb'))